In [1]:
import PyPDF2
import regex as re
from nltk import word_tokenize 
from nltk.util import ngrams
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from deviation_dictionary import *
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import json
import os
import glob
from tqdm import tqdm
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

In [2]:
def get_text(file):
    text = ''
    with open(file, 'rb') as pdfFileObj:
        pdfReader = PyPDF2.PdfReader(pdfFileObj)
        num_pages = len(pdfReader.pages)
        for i in range(num_pages):
            pageObj = pdfReader.pages[i]
            text += pageObj.extract_text()
        text = text.replace('\n', '')
    return text

def get_string_from_list(list):
    string = ''
    for i in range(len(list)):
        string += list[i]
        if i != len(list)-1:
            string += ' '
    return string

def filter(text):
    token = word_tokenize(text)
    token = [word.lower() for word in token]
    token = [word for word in token if word.isalpha()]
    token = [word for word in token if (word == 'down' or not word in stop_words )]
    # remoev single character
    token = [word for word in token if len(word) > 1]
    # remove th
    token = [word for word in token if word != 'th']
    for i in range(len(token)):
        if token[i] == 'per' and token[i+1] == 'cent':
            token[i] = 'percent'
            token[i+1] = ''
    token = [word for word in token if word != '']
    return token

def get_bigram(token):
    token = [stemmer.stem(word) for word in token]
    bigram = ngrams(token, 2)
    return list(bigram)

def get_trigram(token):
    token = [stemmer.stem(word) for word in token]
    trigram = ngrams(token, 3)
    return list(trigram)

In [3]:
# high_dict contains the words, for words with size 1 count them in the bigram list
def count_length_1(bigram_list, term):
    count = 0
    for bigram in bigram_list:
        if bigram[0] == term:
            count+=1
    return count


def count_length_2(bigram_list, term1, term2):
    count = 0
    for i in range(len(bigram_list)):
        if term1 in bigram_list[i]:
            if i-7 < 0:
                for k in range(i+7):
                    if term2 in bigram_list[k]:
                        count += 1
            elif i+7 > len(bigram_list):
                for k in range(i, len(bigram_list)):
                    if term2 in bigram_list[k]:
                        count += 1
            else:
                for k in range(i-7, i+7):
                    if term2 in bigram_list[k]:
                        count += 1
    return count

In [8]:
os.chdir('C:/Users/karti/OneDrive/Desktop/IP/IP/MPC')


# opening each file in each folder
def make_high_low():
    file_names = glob.glob('*.pdf')
    hd = {}
    ld = {}
    for file_name in file_names:
        print(file_name)
        text = get_text(file_name)
        token = filter(text)
        N = len(token)
        string_list = get_string_from_list(token)
        bigram_list = get_bigram(token)
        for i in range(len(high_dict)):
            if(len(high_dict[i].split(' ')) == 1):
                term = high_dict[i]
                term = stemmer.stem(term)
                hd[high_dict[i]] = count_length_1(bigram_list, high_dict[i])
            elif(len(high_dict[i].split(' ')) == 2):
                term1 = high_dict[i]
                term1 = stemmer.stem(term1.split(' ')[0])
                term2 = high_dict[i]
                term2 = stemmer.stem(term2.split(' ')[1])
                hd[high_dict[i]] = count_length_2(bigram_list, term1, term2)
            
        for i in range(len(low_dict)):
            if(len(low_dict[i].split(' ')) == 1):
                term = low_dict[i]
                term = stemmer.stem(term)
                ld[low_dict[i]] = count_length_1(bigram_list, low_dict[i]) 
            elif(len(low_dict[i].split(' ')) == 2):
                term1 = low_dict[i]
                term1 = stemmer.stem(term1.split(' ')[0])
                term2 = low_dict[i]
                term2 = stemmer.stem(term2.split(' ')[1])
                ld[low_dict[i]] = count_length_2(bigram_list, term1, term2)

        high = sum(hd.values())
        low = sum(ld.values())
        score = (high-low)/(high+low)
        print(f"{file_name}, {score}")
        # dump the score , hd and ld into json named file_name
        final_dict = {"high": hd, "low": ld,'score': score}
        file = file_name + '.json'
        with open(file, 'w') as fp:
            json.dump(final_dict, fp, indent = 4)

    return None

In [10]:
make_high_low()

10-2016.pdf
10-2016.pdf, 0.41379310344827586
10-2017.pdf
10-2017.pdf, 0.75
10-2018.pdf
10-2018.pdf, 0.8311688311688312
10-2019.pdf
10-2019.pdf, 0.5714285714285714
10-2020.pdf
10-2020.pdf, -0.3333333333333333
10-2021.pdf
10-2021.pdf, 0.8571428571428571
12-2016.pdf
12-2016.pdf, 0.34782608695652173
12-2017.pdf
12-2017.pdf, 0.7241379310344828
12-2018.pdf
12-2018.pdf, 0.8958333333333334
12-2019.pdf
12-2019.pdf, 0.5294117647058824
12-2020.pdf
12-2020.pdf, 0.6
12-2021.pdf
12-2021.pdf, 0.8490566037735849
12-2022.pdf
12-2022.pdf, 0.8064516129032258
2-2017.pdf
2-2017.pdf, 0.9493670886075949
2-2018.pdf
2-2018.pdf, 0.7754010695187166
2-2019.pdf
2-2019.pdf, 1.0
2-2020.pdf
2-2020.pdf, 0.6666666666666666
2-2021.pdf
2-2021.pdf, 1.0
2-2022.pdf
2-2022.pdf, 0.7222222222222222
2-2023.pdf
2-2023.pdf, 0.9111111111111111
3-2020.pdf
3-2020.pdf, 0.3333333333333333
4-2017.pdf
4-2017.pdf, 0.6610169491525424
4-2018.pdf
4-2018.pdf, 0.9
4-2019.pdf
4-2019.pdf, 0.45454545454545453
4-2020.pdf
4-2020.pdf, 0.73333333333